# Keras Intensive Run

Instructions
1. Define the global variables;
2. Run the second snippet. It builds the network and saves the output to a folder displayed on the console.

In [1]:
# TEST VARIABLES
labelsGroup = [
    ["happy", "sad"],
]
iterationsGroup = [100]
hiddenLayersGroup = [
    [144],
]
batchSize = 128
hyperParametersGroup = [0, 0.01]

In [5]:
# GLOBAL VARIABLES
labelsGroup = [
    # Gonçalo
    ["happy", "neutral"],
    ["angry", "fear", "happy", "neutral", "sad", "surprise"],
    ["fear", "angry"],
    ["happy", "sad"],
    # Margarida
    # ["fear", "sad", "happy"],
    # ["happy", "angry", "neutral"],
    # ["fear", "angry", "surprise"],
    # ["fear", "angry", "surprise", "neutral"],
    
]
iterationsGroup = [200]
hiddenLayersGroup = [
    [20],
    [144],
    [144, 144],
    [300,200,100,50],
    [300,100,100,100],
]
batchSize = 128
hyperParametersGroup = [0, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]

In [6]:
def confusionMatrix(labels, pred, y):
    
    confusionTFPN = {}
    confusionLabels = {}
    
    # Foreach emotion
    for ie in range(len(labels)):
        # TRUE AND FALSE POSITIVES AND NEGATIVES (TFPN)
        
        # Get indexes where emotion was predicted
        ieIndexesPred = [i for i in range(pred.size) if pred[i]==ie]
        ieIndexesNotPred = [i for i in range(pred.size) if pred[i]!=ie]

        # Predicted, and Actual
        TP = sum(pred[:,np.newaxis][ieIndexesPred]==y[ieIndexesPred])
        # Predicted, but not actual
        FP = sum(pred[:,np.newaxis][ieIndexesPred]!=y[ieIndexesPred])
        # Not predicted, but actual
        FN = sum(y[ieIndexesNotPred]==ie)
        # Not predicted and not actual
        TN = sum(y[ieIndexesNotPred]!=ie)

        TP = TP[0] if TP else 0
        FP = FP[0] if FP else 0
        FN = FN[0] if FN else 0
        TN = TN[0] if TN else 0
        
        confusionTFPN[labels[ie]] = {
            'TP': TP,
            'FP': FP,
            'FN': FN,
            'TN': TN
        }
        
        # CONFUSION WITH OTHER labels (Confusion)
        # For emotion e
        # Count the number of predictions made for all classes
        # Foreach emotion, check how many times it has been predicted 
        
        # Get indexes where emotion is real
        ieIndexesY = [i for i in range(y.size) if y[i]==ie]
        
        # Foreach matching prediction, check what emotion was predicted
        confusionLabels[labels[ie]] = {
            e: sum(pred[:,np.newaxis][ieIndexesY]==labels.index(e))[0] for e in labels
        }
        
    return confusionTFPN, confusionLabels

def outputConfusionTables(labels, testExamplesNumber, confusionTFPN, confusionLabels, axs, axsLine, axsTitle):
     # TABLES
    rows = tuple(labels)
    
    # TABLES / TFPN
    # Output confusion matrix as plot table
    data = [
        [
            f"{vals['TP']} ({vals['TP']/testExamplesNumber*100 if vals['TP'] else 0:.2f}%)",
            f"{vals['TN']} ({vals['TN']/testExamplesNumber*100 if vals['TP'] else 0:.2f}%)",
            f"{vals['TP']+vals['TN']} ({(vals['TP']+vals['TN'])/testExamplesNumber*100 if vals['TP']+vals['TN'] else 0:.2f}%)",
            f"{vals['FP']} ({vals['FP']/testExamplesNumber*100 if vals['FP'] else 0:.2f}%)",
            f"{vals['FN']} ({vals['FN']/testExamplesNumber*100 if vals['FN'] else 0:.2f}%)",
            f"{vals['FP']+vals['FN']} ({(vals['FP']+vals['FN'])/testExamplesNumber*100 if vals['FP']+vals['FN'] else 0:.2f}%)",
            vals['FP']+vals['FN']+vals['TP']+vals['TN']
        ]
        for _, vals in confusionTFPN.items()
    ]
    columns = ['TP (%)', 'TN (%)', 'T (%)', 'FP (%)', 'FN (%)', 'F (%)', f"Total"]
    
    # Colorize cells depending on value
    cellColoursTF=plt.cm.Blues([
        [ (vals['TP']+vals['TN'])/testExamplesNumber, (vals['FP']+vals['FN'])/testExamplesNumber ] for _, vals in confusionTFPN.items()
    ])
    cellColourPos=plt.cm.Greens([
        [ vals['TP']/(vals['TP']+vals['FP']) if vals['TP']+vals['FP'] else 0, vals['FP']/(vals['TP']+vals['FP']) if vals['TP']+vals['FP'] else 0 ] for _, vals in confusionTFPN.items()
    ])
    cellColourNeg=plt.cm.Reds([
        [ vals['TN']/(vals['TN']+vals['FN']) if vals['TN']+vals['FN'] else 0, vals['FN']/(vals['TN']+vals['FN']) if vals['TN']+vals['FN'] else 0 ] for _, vals in confusionTFPN.items()
    ])
    
    cellColours = [
        [
            cellColourPos[i][0],
            cellColourNeg[i][0],
            cellColoursTF[i][0],
            cellColourPos[i][1],
            cellColourNeg[i][1],
            cellColoursTF[i][1],
            [0, 0, 0, 0]
        ]
        for i in range(len(rows))
    ]
        
    the_table = axs[axsLine].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    # the_table.scale(1.2, 1)
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(20)
    axs[axsLine].axis('off')
    axs[axsLine].axis('tight')
    axs[axsLine].set_title(f"Confusion matrix {axsTitle}", fontsize=20, pad=0)    
    
    # TABLES / Confusion    
    # Output confusion matrix as plot table
    data = [[o for _,o in others.items()] for _,others in confusionLabels.items()]
    columns = rows
    
    # Colorize cells depending on value
    vals = [o for _,others in confusionLabels.items() for _,o in others.items()]
    normal = plt.Normalize(min(vals)-1, max(vals)+1)
    cellColours=plt.cm.Blues(normal(data))
        
    the_table = axs[axsLine+1].table(
      cellText=data,
      rowLabels=rows,
      colLabels=columns,
      loc='center',
      cellColours=cellColours
    )
    
    the_table.set_fontsize(20)
    # the_table.scale(1, 4)
    axs[axsLine+1].axis('off')
    axs[axsLine+1].axis('tight')
    axs[axsLine+1].set_title(f"True/Predicted {axsTitle}", fontsize=20)

In [ ]:
#### IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#to load matlab mat files
from scipy.io import loadmat
# for saving metrics
import os 
import json
# for neural network
import tensorflow as tf
from tensorflow import keras

if not os.path.exists('./metrics'):
    os.makedirs('./metrics')

# FOR EACH LABEL GROUP
for labelNames in labelsGroup:
    # LOAD FILES
    print("\n\nLABELS", ', '.join(labelNames))

    # Load Training/Dev/Test data
    mat=loadmat(f"../datasets/train/{'_'.join(labelNames)}.mat")
    X, y = mat["X"], mat["y"]
    matDev=loadmat(f"../datasets/dev/{'_'.join(labelNames)}.mat")
    X_valid, y_valid = matDev["X"], matDev["y"]
    matTest=loadmat(f"../datasets/test/{'_'.join(labelNames)}.mat")
    X_test, y_test = matTest["X"], matTest["y"]
    
    # NUMBERS
    m = X.shape[0] # number of training examples
    labels = np.max(y)+1 # number of labels
    features = X.shape[1] # number of features per example

    # Output user feedback
    print(f"Loaded {m} traing examples with {labels} labels, each with {features} features (pixels).")
    print(f"Loaded {X_valid.shape[0]} dev examples.")
    print(f"Loaded {X_test.shape[0]} test examples.")
    
    print()
    
    # Create folder for metrics
    labelFolder = f"./metrics/{'_'.join(labelNames)}"
    if not os.path.exists(labelFolder):
        os.makedirs(labelFolder)
    if not os.path.exists(f"{labelFolder}/history"):
        os.makedirs(f"{labelFolder}/history")
    if not os.path.exists(f"{labelFolder}/predict"):
        os.makedirs(f"{labelFolder}/predict")
    if not os.path.exists(f"{labelFolder}/archive"):
        os.makedirs(f"{labelFolder}/archive")
    
    # FOREACH HIDDEN LAYER
    for hiddenLayers in hiddenLayersGroup:
        # FOR EACH HYPER PARAMETER
        for hyperParameter in hyperParametersGroup:
            # FOR EACH ITERATION NUMBER
            for iterations in iterationsGroup:
                
                hiddenLayersText = '_'.join(map(str, hiddenLayers)) if len(hiddenLayers)>1 else hiddenLayers[0]
                print(hiddenLayersText)
                combinationName = f"{iterations}iter_{batchSize}batchS_{hyperParameter}hyper_{len(hiddenLayers)}hlayers__{hiddenLayersText}"
                folder = f"{labelFolder}/archive/{combinationName}/"
                if not os.path.exists(folder):
                    os.makedirs(folder)

                print(f"\nBuilding network for {iterations} iterations and batch size of {batchSize} and {len(hiddenLayers)} hidden layers: {hiddenLayersText}...")
                print("With regularization!" if hyperParameter else "Without regularization!")
                print()

                # BUILD NETWORK
                # Create model
                modelSeq = []
                # Flattens each image (48x48) to 1x2304
                modelSeq.append(keras.layers.Flatten(input_shape = [2304]))
                # Hidden layers with relu activation function
                for h in hiddenLayers:                        
                    modelSeq.append(keras.layers.Dense(
                        h, 
                        activation = "relu", 
                        bias_regularizer= keras.regularizers.l2(hyperParameter) if hyperParameter else None
                    ))
                # Output layer with softmax activation function
                modelSeq.append(keras.layers.Dense(
                    labels, 
                    activation = "softmax",
                    bias_regularizer= keras.regularizers.l2(hyperParameter) if hyperParameter else None
                ))

                model = keras.models.Sequential(modelSeq)

                # Compile model
                model.compile(
                    # Using sparse categorical crossentropy loss function
                    loss = "sparse_categorical_crossentropy",
                    # Using stochastic gradient descent as gradient descent
                    optimizer = "sgd",
                    # In addition to cost, we want accuracy to help understanding how the model is working 
                    metrics = ["accuracy"]
                )

                # Train the network
                history = model.fit(
                    X,
                    y,
                    epochs = iterations,
                    batch_size = batchSize,
                    validation_data = (X_valid, y_valid)
                )

                # METRICS (SAVE TO FILE)

                # Model training history
                with open(f"{folder}/history.json", "w") as f:
                    json.dump(history.history, f)

                # Test model evaluation
                with open(f"{folder}/evaluation.json", "w") as f:
                    json.dump(model.evaluate(X_test, y_test, return_dict=True), f)

                # Prediction for test data
                with open(f"{folder}/predict.json", "w") as f:
                    json.dump(model.predict(X_test).tolist(), f)

                print(f"\nModel trained and metrics saved to {folder}!")

                # GENERATE GRAPHS AND SAVE TO FILE

                # Model training history
                pd.DataFrame(history.history).plot(figsize = (16, 10))
                plt.grid(True)
                plt.gca().set_ylim(0, 1)
                plt.title("Neural network training metrics")
                plt.savefig(f"{labelFolder}/history/{combinationName}.png")

                # Prediction for test data
                predict = model.predict(X_test)
                y_pred = np.array([np.argmax(p) for p in predict])
                confusionTFPN, confusionLabels = confusionMatrix(labelNames, y_pred, y_test)
                fig, axs = plt.subplots(2,1)
                outputConfusionTables(labelNames, y_test.size, confusionTFPN, confusionLabels, axs, 0, "WITHOUT regularization")
                fig.set_size_inches(18,6*len(labelNames))
                fig.subplots_adjust(left=0.2, top=20)
                fig.tight_layout()
                fig.savefig(f"{labelFolder}/predict/{combinationName}.png")



LABELS happy, neutral
Loaded 2000 traing examples with 2 labels, each with 2304 features (pixels).
Loaded 400 dev examples.
Loaded 400 test examples.

20

Building network for 200 iterations and batch size of 128 and 1 hidden layers: 20...
Without regularization!

Epoch 1/200
16/16 [==============================] - 0s 9ms/step - loss: 0.7724 - accuracy: 0.5089 - val_loss: 0.7055 - val_accuracy: 0.5200
Epoch 2/200
16/16 [==============================] - 0s 4ms/step - loss: 0.7027 - accuracy: 0.5345 - val_loss: 0.6683 - val_accuracy: 0.6100
Epoch 3/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6713 - accuracy: 0.5773 - val_loss: 0.6586 - val_accuracy: 0.5825
Epoch 4/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6611 - accuracy: 0.6187 - val_loss: 0.7241 - val_accuracy: 0.5025
Epoch 5/200
16/16 [==============================] - 0s 5ms/step - loss: 0.6879 - accuracy: 0.5595 - val_loss: 0.6596 - val_accuracy: 0.5725
Epoch 6/200
16/16 [=========

Epoch 57/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5572 - accuracy: 0.7112 - val_loss: 0.5595 - val_accuracy: 0.7400
Epoch 58/200
16/16 [==============================] - 0s 5ms/step - loss: 0.5593 - accuracy: 0.7120 - val_loss: 0.5574 - val_accuracy: 0.7275
Epoch 59/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5479 - accuracy: 0.7302 - val_loss: 0.5580 - val_accuracy: 0.7275
Epoch 60/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5701 - accuracy: 0.6926 - val_loss: 0.5856 - val_accuracy: 0.6600
Epoch 61/200
16/16 [==============================] - 0s 5ms/step - loss: 0.5399 - accuracy: 0.7373 - val_loss: 0.5572 - val_accuracy: 0.7250
Epoch 62/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5304 - accuracy: 0.7401 - val_loss: 0.5537 - val_accuracy: 0.7400
Epoch 63/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5445 - accuracy: 0.7413 - val_loss: 0.5553 - val_accuracy: 0.7325
Epoch 

16/16 [==============================] - 0s 4ms/step - loss: 0.5571 - accuracy: 0.7099 - val_loss: 0.5494 - val_accuracy: 0.7325
Epoch 115/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4998 - accuracy: 0.7526 - val_loss: 0.5835 - val_accuracy: 0.6875
Epoch 116/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4971 - accuracy: 0.7500 - val_loss: 0.6432 - val_accuracy: 0.6350
Epoch 117/200
16/16 [==============================] - 0s 6ms/step - loss: 0.5131 - accuracy: 0.7533 - val_loss: 0.6924 - val_accuracy: 0.6175
Epoch 118/200
16/16 [==============================] - 0s 5ms/step - loss: 0.5172 - accuracy: 0.7408 - val_loss: 0.6813 - val_accuracy: 0.6175
Epoch 119/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4988 - accuracy: 0.7617 - val_loss: 0.5496 - val_accuracy: 0.7325
Epoch 120/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4997 - accuracy: 0.7554 - val_loss: 0.5614 - val_accuracy: 0.7375
Epoch 121/200

16/16 [==============================] - 0s 4ms/step - loss: 0.4893 - accuracy: 0.7568 - val_loss: 0.5632 - val_accuracy: 0.7075
Epoch 172/200
16/16 [==============================] - 0s 5ms/step - loss: 0.4547 - accuracy: 0.7915 - val_loss: 0.5485 - val_accuracy: 0.7275
Epoch 173/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4509 - accuracy: 0.7972 - val_loss: 0.5613 - val_accuracy: 0.7150
Epoch 174/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4479 - accuracy: 0.7921 - val_loss: 0.6282 - val_accuracy: 0.6475
Epoch 175/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5066 - accuracy: 0.7493 - val_loss: 0.5607 - val_accuracy: 0.7100
Epoch 176/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4964 - accuracy: 0.7567 - val_loss: 0.5491 - val_accuracy: 0.7325
Epoch 177/200
16/16 [==============================] - 0s 4ms/step - loss: 0.4549 - accuracy: 0.7824 - val_loss: 0.5460 - val_accuracy: 0.7375
Epoch 178/200

16/16 [==============================] - 0s 4ms/step - loss: 0.6229 - accuracy: 0.6608 - val_loss: 0.6055 - val_accuracy: 0.7375
Epoch 27/200
16/16 [==============================] - 0s 5ms/step - loss: 0.6088 - accuracy: 0.6679 - val_loss: 0.6102 - val_accuracy: 0.7025
Epoch 28/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6193 - accuracy: 0.6696 - val_loss: 0.6008 - val_accuracy: 0.7275
Epoch 29/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6086 - accuracy: 0.6789 - val_loss: 0.6026 - val_accuracy: 0.7075
Epoch 30/200
16/16 [==============================] - 0s 4ms/step - loss: 0.6126 - accuracy: 0.6621 - val_loss: 0.6212 - val_accuracy: 0.6350
Epoch 31/200
16/16 [==============================] - 0s 4ms/step - loss: 0.5989 - accuracy: 0.7070 - val_loss: 0.6219 - val_accuracy: 0.6350
Epoch 32/200
16/16 [==============================] - 0s 5ms/step - loss: 0.6351 - accuracy: 0.6469 - val_loss: 0.5974 - val_accuracy: 0.6875
Epoch 33/200
16/16 

...

Epoch 99/200
16/16 [==============================] - 0s 6ms/step - loss: 0.5383 - accuracy: 0.7268 - val_loss: 0.5368 - val_accuracy: 0.7250
Epoch 100/200
16/16 [==============================] - 0s 6ms/step - loss: 0.4820 - accuracy: 0.7637 - val_loss: 0.5352 - val_accuracy: 0.7200
Epoch 101/200
16/16 [==============================] - 0s 7ms/step - loss: 0.4749 - accuracy: 0.7732 - val_loss: 0.5697 - val_accuracy: 0.6850
Epoch 102/200
16/16 [==============================] - 0s 6ms/step - loss: 0.4761 - accuracy: 0.7847 - val_loss: 0.5435 - val_accuracy: 0.7500
Epoch 103/200
16/16 [==============================] - 0s 6ms/step - loss: 0.4987 - accuracy: 0.7619 - val_loss: 0.5381 - val_accuracy: 0.7375
Epoch 104/200
16/16 [==============================] - 0s 7ms/step - loss: 0.4996 - accuracy: 0.7578 - val_loss: 0.5433 - val_accuracy: 0.7425
Epoch 105/200
16/16 [==============================] - 0s 7ms/step - loss: 0.4729 - accuracy: 0.7786 - val_loss: 0.5342 - val_accuracy: 0.7275


16/16 [==============================] - 0s 10ms/step - loss: 0.4230 - accuracy: 0.8188 - val_loss: 0.5308 - val_accuracy: 0.7275
Epoch 157/200
16/16 [==============================] - 0s 11ms/step - loss: 0.4708 - accuracy: 0.7705 - val_loss: 0.5482 - val_accuracy: 0.7450
Epoch 158/200
16/16 [==============================] - 0s 10ms/step - loss: 0.4949 - accuracy: 0.7423 - val_loss: 0.5275 - val_accuracy: 0.7250
Epoch 159/200
16/16 [==============================] - 0s 10ms/step - loss: 0.4438 - accuracy: 0.7955 - val_loss: 0.5288 - val_accuracy: 0.7300
Epoch 160/200
16/16 [==============================] - 0s 9ms/step - loss: 0.4274 - accuracy: 0.8071 - val_loss: 0.5662 - val_accuracy: 0.6800
Epoch 161/200
16/16 [==============================] - 0s 9ms/step - loss: 0.4423 - accuracy: 0.8027 - val_loss: 0.5587 - val_accuracy: 0.7375
Epoch 162/200
16/16 [==============================] - 0s 8ms/step - loss: 0.4374 - accuracy: 0.8125 - val_loss: 0.5955 - val_accuracy: 0.6475
Epoch 163

/home/goncalom/anaconda3/lib/python3.8/site-packages/pandas/plotting/_matplotlib/core.py:328: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


144

Building network for 200 iterations and batch size of 128 and 1 hidden layers: 144...
With regularization!

Epoch 1/200
16/16 [==============================] - 1s 19ms/step - loss: 0.7700 - accuracy: 0.4982 - val_loss: 0.7603 - val_accuracy: 0.4975
Epoch 2/200
16/16 [==============================] - 0s 6ms/step - loss: 0.7425 - accuracy: 0.5004 - val_loss: 0.7477 - val_accuracy: 0.4950
Epoch 3/200
16/16 [==============================] - 0s 7ms/step - loss: 0.6867 - accuracy: 0.5508 - val_loss: 0.7098 - val_accuracy: 0.5200
Epoch 4/200
16/16 [==============================] - 0s 7ms/step - loss: 0.6863 - accuracy: 0.5658 - val_loss: 0.7227 - val_accuracy: 0.5025
Epoch 5/200
16/16 [==============================] - 0s 6ms/step - loss: 0.6567 - accuracy: 0.6079 - val_loss: 0.6578 - val_accuracy: 0.6175
Epoch 6/200
16/16 [==============================] - 0s 7ms/step - loss: 0.6521 - accuracy: 0.6095 - val_loss: 0.6610 - val_accuracy: 0.5850
Epoch 7/200
16/16 [=====================

Epoch 58/200
16/16 [==============================] - 0s 8ms/step - loss: 0.5415 - accuracy: 0.7167 - val_loss: 0.5658 - val_accuracy: 0.7000
Epoch 59/200
16/16 [==============================] - 0s 8ms/step - loss: 0.5582 - accuracy: 0.7146 - val_loss: 0.6124 - val_accuracy: 0.6475
Epoch 60/200
16/16 [==============================] - 0s 8ms/step - loss: 0.5853 - accuracy: 0.6813 - val_loss: 0.5650 - val_accuracy: 0.7050
Epoch 61/200
16/16 [==============================] - 0s 8ms/step - loss: 0.5289 - accuracy: 0.7483 - val_loss: 0.5963 - val_accuracy: 0.6525
Epoch 62/200
16/16 [==============================] - 0s 8ms/step - loss: 0.5636 - accuracy: 0.7054 - val_loss: 0.5633 - val_accuracy: 0.7000
Epoch 63/200
16/16 [==============================] - 0s 7ms/step - loss: 0.5345 - accuracy: 0.7347 - val_loss: 0.5768 - val_accuracy: 0.6950
Epoch 64/200
16/16 [==============================] - 0s 8ms/step - loss: 0.5381 - accuracy: 0.7344 - val_loss: 0.6064 - val_accuracy: 0.6600
Epoch 